<a href="https://colab.research.google.com/github/ErikHartman/bopep/blob/main/bopep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BoPep: identifying peptide binders in large scale peptidomic data

Bayesian optimization guided search for binders in large scale peptidomic datasets.

Relies on ESM2 for peptide embeddings, ColabFold utilizing AlphaFold 2 multimer v3 for docking, and PyRosetta for interface quality calculations. A deep ensemble is used as a surrogate model utilizing Torch. 

Note that the use of PyRosetta for **commercial** purposes requires purchasing a license.

Set runtime to T4 GPU (Runtime > Change runtime type).

In [ ]:
#@title Installation

import os
import sys
from pathlib import Path
import shutil

# Set cache directory to /content to ensure we have write permissions
os.environ['XDG_CACHE_HOME'] = '/content'

# Check Python version
PYTHON_VERSION = f"{sys.version_info.major}.{sys.version_info.minor}"
print(f"Using Python version: {PYTHON_VERSION}")

# Install ColabFold
try:
    import colabfold
    print("ColabFold is already installed.")
except ImportError:
    print("Installing ColabFold...")
    !pip install -q --no-warn-conflicts 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'

# Install JAX with CUDA support
try:
    import jax
    if jax.local_devices()[0].platform == 'gpu':
        print("JAX is already installed with CUDA support.")
    else:
        print("Reinstalling JAX with CUDA support...")
        !pip uninstall -y jax jaxlib
        !pip install -q --no-warn-conflicts --upgrade dm-haiku==0.0.10 'jax[cuda12_pip]'==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
except ImportError:
    print("Installing JAX with CUDA support...")
    !pip uninstall -y jax jaxlib
    !pip install -q --no-warn-conflicts --upgrade dm-haiku==0.0.10 'jax[cuda12_pip]'==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# Install HH-suite via apt-get
if shutil.which("hhsearch") is not None:
    print("HH-suite is already installed.")
else:
    print("Installing HH-suite...")
    !apt-get update
    !apt-get install -y hhsuite

# Install Kalign via apt-get (required for MSA generation)
if shutil.which("kalign") is not None:
    print("Kalign is already installed.")
else:
    print("Installing Kalign...")
    !apt-get install -y kalign

# Install OpenMM and PDBFixer via pip
try:
    import openmm
    import pdbfixer
    print("OpenMM and PDBFixer are already installed.")
except ImportError:
    print("Installing OpenMM and PDBFixer...")
    !pip install --quiet openmm
    pdbfixer_repo_url = "https://github.com/openmm/pdbfixer.git"
    pdbfixer_dir = "/content/pdbfixer"

    if not os.path.exists(pdbfixer_dir):
        print("Cloning the PDBFixer repository...")
        !git clone {pdbfixer_repo_url} {pdbfixer_dir}
    else:
        print("PDBFixer repository already cloned.")

    print("Installing PDBFixer...")
    !pip install {pdbfixer_dir}

# Fetch bopep repository
if not os.path.exists("/content/bopep"):
    print("Fetching bopep...")
    !git clone https://github.com/ErikHartman/bopep /content/bopep/
    print("Installing necessary packages using pip...")
    !pip install -r /content/bopep/requirements.txt --quiet
else:
    print("bopep repository already exists.")

# Install ESM model
esm_model_path = "/root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt"
if not os.path.exists(esm_model_path):
    print("Installing fair-esm...")
    !pip install --quiet fair-esm
    print("Downloading ESM model...")
    import esm
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
else:
    print("ESM model already exists.")

# Download AlphaFold model parameters
params_dir = Path("/root/.cache/colabfold/")
expected_param_files = [
    params_dir / f"params_model_{model_num}_multimer_v3.npz"
    for model_num in range(1, 6)
]
if all(param_file.exists() for param_file in expected_param_files):
    print("AlphaFold model parameters already downloaded.")
else:
    !mkdir /root/.cache/colabfold/
    !mkdir /root/.cache/colabfold/params
    print("Downloading AlphaFold model parameters...")
    from colabfold.download import download_alphafold_params
    download_alphafold_params("alphafold2_multimer_v3", params_dir)
    !scp /root/.cache/colabfold /content

# Install PyRosetta
try:
    import pyrosetta
    print("PyRosetta is already installed.")
except ImportError:
    try:
        import pyrosettacolabsetup
    except ImportError:
        print("Installing pyrosettacolabsetup...")
        !pip install pyrosettacolabsetup
    print("Installing PyRosetta...")
    import pyrosettacolabsetup
    pyrosettacolabsetup.install_pyrosetta(serialization=True, cache_wheel_on_google_drive=False)


Fetching bopep
Cloning into '/content/bopep'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 96 (delta 42), reused 57 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (96/96), 107.15 KiB | 5.64 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Installing necessary packages using pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


Installing PyRosetta

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels.serialization...

Resolving west.rosettacommons.org (west.rosettacommons.org)... 128.95.160.153, 2607:4000:406::160:153

HTTP request sent, awaiting response... 302 Found
Location: https://west.rosettacommons.org/pyrosetta/release/release/PyRosetta4.Release.python310.ubuntu.cxx11thread.serialization.wheel/pyrosetta-2024.42+release.3366cf78a3-cp310-cp310-linux_x86_64.whl [following]
--2024-11-13 08:31:49--  https://west.rosettacommons.org/pyrosetta/release/release/PyRosetta4.Release.python310.ubuntu.cxx11thread.serialization.wheel/pyrosetta-2024.42+release.3366cf78a3-cp310-cp310-linux_x86_64.whl

HTTP request sent, awaiting response... 200 OK
Length: 1576918002 (1.5G)
--2024-11-13 08:31:49--  ht

In [ ]:
#@title Embedding Settings
import pandas as pd

%cd bopep

# Data input
# @markdown Upload your input data file and set the path.
data_file = "/content/bopep/data/test_data.csv" #@param {type:"string"}

if not os.path.exists(data_file):
  raise ValueError("The data file does not exist in the path.")

data = pd.read_csv(data_file)  # Load the CSV file
peptides = data["peptide"].tolist()
# @markdown  ### Filtering options:

# @markdown Set maximum and minimum peptide length
max_length = 30  #@param {type:"slider", min:10, max:60, step:1}
min_length = 5   #@param {type:"slider", min:1, max:30, step:1}

# @markdown Set maximum repeat length for amino acids
max_repeat_length = 5  #@param {type:"slider", min:1, max:15, step:1}

# @markdown  Set maximum allowed fraction of single amino acids
max_single_aa_fraction = 0.73  #@param {type:"slider", min:0, max:1, step:0.01}

# @markdown  Variance kept during PCA reduction
pca_variance = 0.95  #@param {type:"slider", min:0.1, max:1, step:0.01}


/content/bopep


In [ ]:
#@title Generate Embeddings
from src.embeddings.embed import embed
from src.embeddings.utils import filter_peptides
from sklearn.decomposition import PCA
import numpy as np

filtered_peptides = filter_peptides(peptides, max_single_aa_fraction, max_repeat_length, min_length, max_length)
embeddings = embed(filtered_peptides, model_path=esm_model_path)

if pca_variance < 1:
  embedding_array = np.array(list(embeddings.values()))
  peptide_sequences = list(embeddings.keys())
  pca = PCA(n_components=0.95, svd_solver="full")
  embeddings_reduced = pca.fit_transform(embedding_array)
  print(f"Reduced embedding size: {np.shape(embeddings_reduced)} (before PCA: {np.shape(embedding_array)})")
  embeddings = {
      peptide_sequences[i]: embeddings_reduced[i] for i in range(len(peptide_sequences))
  }


/usr/local/lib/python3.10/dist-packages/esm/pretrained.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_data = torch.load(str(model_location), map_location="cpu")


Model moved to GPU.


Generating embeddings: 100%|██████████| 16/16 [00:12<00:00,  1.28it/s]


Reduced embedding size: (1000, 96) (before PCA: (1000, 1280))


In [ ]:
#@title Bayesian Optimization Settings

from src.docking.dock_peptides import extract_sequence_from_pdb

# Target structure file (PDB format)
target_structure = "/content/bopep/data/4glp.pdb"  #@param {type:"string"}
target_sequence = extract_sequence_from_pdb(target_structure, "A")

# @markdown Model settings
num_recycles = 9  #@param {type:"slider", min:1, max:20, step:1}  # Number of recycles for AlphaFold
num_relax = 1  #@param {type:"slider", min:0, max:5, step:1}  # Number of relaxations
num_models = 5  #@param {type:"slider", min:1, max:5, step:1}  # Number of models
num_processes = 2  #@param {type:"slider", min:1, max:8, step:1}  # Number of CPU processes for docking
gpu_ids = ["0"]  #@param {type:"hidden"}  # List of GPU IDs, Colab generally has one GPU
relax_max_iterations = 200 #@param [0, 200, 2000] {type:"raw"}

# @markdown Stopping and relaxation parameters
recycle_early_stop_tolerance = 0.3  #@param {type:"slider", min:0, max:1, step:0.1}  # Early stop tolerance
amber = True  #@param {type:"boolean"}  # Whether to use AMBER for relaxation

# @markdown Target binding site (optional)
binding_site_residue_indices = [44, 49, 74, 82, 89, 105]  #@param {type:"raw"}  # Binding site residues

# @markdown Objective weights for Bayesian optimization
iptm_score_weight = 0.5  #@param {type:"number"}
interface_sasa_weight = 0.1  #@param {type:"number"}
interface_dG_weight = 0.1  #@param {type:"number"}
rosetta_score_weight = 0.1  #@param {type:"number"}
interface_delta_hbond_unsat_weight = 0.1  #@param {type:"number"}
packstat_weight = 0.1  #@param {type:"number"}

# @markdown Bayesian Optimization Iterations
n_initial = 30  #@param {type:"slider", min:50, max:1000, step:1}  # Initial number of evaluations
n_exploration_iterations = 10  #@param {type:"slider", min:50, max:1500, step:1}  # Number of exploration iterations
n_exploitation_with_distance_weight = 10  #@param {type:"slider", min:50, max:3000, step:50}  # Exploitation iterations with distance weight
n_exploitation_iterations = 0  #@param {type:"slider", min:0, max:1000, step:1}  # Number of exploitation iterations without distance weight
batch_size = 4  #@param {type:"slider", min:1, max:32, step:1}  # Batch size for optimization
agreeing_models = 0  #@param {type:"slider", min:0, max:10, step:1}  # Number of agreeing models to use
proximity_threshold = 5.0  #@param {type:"slider", min:1, max:20, step:0.5}  # Proximity threshold in Ångstroms
hparam_opt_interval = 10  #@param {type:"slider", min:10, max:200, step:10}  # Hyperparameter optimization interval

In [ ]:
#@title Hyperparameter Optimization Settings

# @markdown Number of layers
n_layers_min = 1  #@param {type:"slider", min:1, max:10, step:1}
n_layers_max = 5  #@param {type:"slider", min:1, max:10, step:1}
n_layers_range = (n_layers_min, n_layers_max)

# @markdown Units in the first layer (log scale)
n_units_l1_min = 32  #@param {type:"slider", min:32, max:4096, step:32}
n_units_l1_max = 1024  #@param {type:"slider", min:32, max:4096, step:32}
n_units_l1_range = (n_units_l1_min, n_units_l1_max)

# @markdown Alpha range (log scale)
alpha_min = 1e-5  #@param {type:"number"}
alpha_max = 1e-3  #@param {type:"number"}
alpha_range = (alpha_min, alpha_max)

# @markdown Learning rate initialization range (log scale)
learning_rate_init_min = 1e-4  #@param {type:"number"}
learning_rate_init_max = 1e-2  #@param {type:"number"}
learning_rate_init_range = (learning_rate_init_min, learning_rate_init_max)

# @markdown Number of trials
n_trials = 50  #@param {type:"slider", min:10, max:200, step:10}

# @markdown Neural network training settings
max_iter = 3000  #@param {type:"number"}
tol = 1e-4  #@param {type:"number"}
n_iter_no_change = 10  #@param {type:"number"}

# @markdown Hidden layer adjustments
hidden_layer_decrease_factor = 2  #@param {type:"slider", min:1, max:4, step:1}
min_hidden_layer_size = 8  #@param {type:"slider", min:4, max:64, step:1}


In [ ]:
#@title Run BoPep!

import pyrosetta

os.makedirs("/content/output", exist_ok=True)

pyrosetta.init('-mute all')

from src.run import run_bayesian_optimization

run_bayesian_optimization(
    embeddings,
    target_structure,
    target_sequence,

    # Docking parameters
    num_recycles=1,
    num_models=1,
    recycle_early_stop_tolerance=0.3,
    amber=True,
    binding_site_residue_indices=None,
    relax_max_iterations=relax_max_iterations,

    # BO parameters
    iptm_score_weight=iptm_score_weight,
    interface_sasa_weight=interface_sasa_weight,
    interface_dG_weight=interface_dG_weight,
    rosetta_score_weight=rosetta_score_weight,
    interface_delta_hbond_unsat_weight=interface_delta_hbond_unsat_weight,
    packstat_weight=packstat_weight,
    n_initial=n_initial,
    n_exploration_iterations=n_exploration_iterations,
    n_exploitation_iterations=n_exploitation_iterations,
    batch_size=batch_size,
    agreeing_models=agreeing_models,
    proximity_threshold=proximity_threshold,
    hparam_opt_interval=hparam_opt_interval,

    # Hyperparameter optimization parameters
    n_layers_range=n_layers_range,
    n_units_l1_range=n_units_l1_range,
    alpha_range=alpha_range,
    learning_rate_init_range=learning_rate_init_range,
    n_splits=5,
    random_state=42,
    n_trials=n_trials,
    n_jobs=1,
    pruner_n_warmup_steps=5,
    direction="maximize",
    pruner_type="MedianPruner",
    sampler_type="TPESampler",
    tol=tol,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=n_iter_no_change,
    hidden_layer_decrease_factor=hidden_layer_decrease_factor,
    min_hidden_layer_size=min_hidden_layer_size,
    bin_edges=None,
)
